## Import of the libraries

In [1]:
import numpy as np
from keras.models import Sequential, Model
from keras import layers, Input
from keras.optimizers import Adam
from keras import backend as K
import tensorflow as tf
import pybobyqa
import scipy.optimize

Using TensorFlow backend.


# (I) Long-term dedicated capacity forecasting

## Function to perform the forecasting

In [ ]:
def forecast(model, data, min_index, max_index, T_l,
                     lookback, num_services, num_iters):
    load_forecasted = np.zeros((int(
        np.ceil((max_index - (min_index + lookback))/T_l)),
                                num_services, num_iters))
    for idx, i in enumerate(range(min_index+lookback, max_index, T_l)):
        block = data[i-lookback:i].reshape((1, lookback, data.shape[1],
                                            data.shape[2], num_services))
        for sample in range(num_iters):
            load_forecasted[idx, :, sample] = model.predict(block)
    return load_forecasted

## Loss function

In [ ]:
def cost_block_1(k_o, k_r, T_l):
    def cost_func_slopes(y_true, y_pred):
        y_pred = tf.expand_dims(y_pred, axis=1)
        y_pred = tf.tile(y_pred, (1, T_l, 1))
        diff = y_pred - y_true
        cost = np.zeros(diff.shape)
        pos_penalty = k_o * diff
        neg_penalty = k_r * diff
        cost = tf.where(diff > 0, pos_penalty, cost)
        cost = tf.where(diff <= 0, neg_penalty, cost)
        cost = tf.abs(cost)
        cost = K.sum(K.sum(cost, axis=-1), axis=-1)
        return cost
    return cost_func_slopes

## Build NN model

In [ ]:
def make_block_1_model(load_input_shape, lookback, num_services, k_o, k_r, T_l):
    '''Build block 1 NN model'''
    inputs = Input((lookback, load_input_shape[1], load_input_shape[2],
                    num_services))
    model = layers.Conv3D(32, (3, 3, 3), activation='relu',
                          padding='same')(inputs)
    model = layers.Conv3D(32, (6, 6, 6), activation='relu',
                          padding='same')(model)
    model = layers.Dropout(0.3)(model, training=True) # adding training=True the Dropout layer will work also with predict
    model = layers.Conv3D(16, (6, 6, 6), activation='relu',
                          padding='same')(model)
    model = layers.Dropout(0.3)(model, training=True)
    model = layers.Flatten()(model)
    model = layers.Dense(64, activation='relu')(model)
    model = layers.Dense(32, activation='relu')(model)
    output = layers.Dense(num_services, activation='relu')(model)
    model = Model(inputs, output)
    model.compile(optimizer=Adam(0.0005), loss=cost_block_1(k_o, k_r, T_l))
    return model

## Create, training and forecast Block (I)

In [ ]:
N = 6
batch_size = 128
num_iters # number of time you repeat the same prediction to measure NN uncertainty
load_input # numpy matrix of shape(len(dataset), nº row, nº col, num_services). 
load_output # numpy matrix of shape (len(dataset), 1, num_services).
model = make_block_1_model(load_input.shape, N, num_services, k_o, k_r, T_l)
history = model.fit_generator(train_gen, steps_per_epoch=150, epochs=100, validation_data=val_gen, validation_steps=40, verbose=0)
load_predicted = forecast(model, load_input, min_index=start_index_test_dataset, max_index=end_index_test_dataset, delay=T_l, lookback=N, num_services=num_services, num_iters=num_iters)

# (II) Long-term total shared capacity forecasting

## Function to perform the forecasting

In [ ]:
def forecast_cluster(model, data, min_index, max_index, T_l, lookback, num_iters):
    load_forecasted = np.zeros((int(np.ceil((max_index - (min_index + lookback))/T_l)), 1, num_iters))
    for idx, i in enumerate(range(min_index+lookback, max_index, T_l)):
        block = data[i-lookback:i].reshape((1, lookback))
        for sample in range(num_iters):
            load_forecasted[idx, :, sample] = model.predict(block)
    return load_forecasted

## Loss function

In [ ]:
def cost_block_2(k_s, k_o, T_l):
    def cost_func(y_true, y_pred):
        epsilon = 0.1
        y_pred = tf.expand_dims(y_pred, axis=1)
        y_pred = tf.tile(y_pred, (1, T_l, 1))
        diff = y_pred - y_true
        cost = np.zeros(diff.shape)
        y1 = -epsilon * diff + k_s
        y2 = -np.true_divide(1, epsilon) * diff + k_s
        y3 = k_o * diff - epsilon*k_s*k_o
        cost = tf.where(diff < 0, y1, cost)
        cost = tf.where(tf.logical_and((diff <= epsilon*k_s), (diff >= 0)), y2, cost)
        cost = tf.where(diff > epsilon*k_s, y3, cost)
        cost = tf.abs(cost)
        cost = K.sum(K.sum(cost, axis=-1), axis=-1)
        return cost
    return cost_func

## Build NN model

In [ ]:
def make_block_2_model(lookback, k_s, k_o, T_l):
    inputs = Input((lookback,1))
    model = layers.Dense(128, activation='relu')(inputs)
    model = layers.Dropout(0.2)(model, training=True)
    model = layers.Dense(64, activation='relu')(model)
    output = layers.Dense(1, activation='relu')(model)
    model = Model(inputs, output)
    model.compile(optimizer=Adam(0.0005),
                  loss=cost_block_2(k_s, k_o, T_l))
    return model

## Create, training and forecast Block (II)

In [ ]:
N = 6
batch_size = 128
num_iters # number of time you repeat the same prediction to measure NN uncertainty
load_input # numpy matrix of shape(len(dataset), 1). 
load_output # numpy matrix of shape (len(dataset), 1).
model_2 = make_block_2_model(N, k_s, k_o, T_l)
history = model_2.fit_generator(train_gen, steps_per_epoch=150, epochs=100, validation_data=val_gen, validation_steps=40, verbose=0)
load_predicted = forecast_block_2(model_2, load_input, min_index=start_index_test_dataset, max_index=end_index_test_dataset, T_l=T_l, lookback=N, num_iters=num_iters)

# (Helper) Short-term demand prediction

## Function to perform the forecasting

In [ ]:
def forecast_batch(model, data, min_index, max_index, lookback, num_services, num_iters):
    load_forecasted = np.zeros((max_index - (min_index + lookback), num_services, num_iters))
    rows = np.arange(min_index+lookback, max_index)
    samples = np.zeros((len(rows), lookback, data.shape[1], data.shape[2], num_services))
    for j, row in enumerate(rows):
        indices = np.arange(row - lookback, row)
        samples[j] = data[indices]
    for sample in range(num_iters):
        load_forecasted[:, :, sample] = model.predict(samples)
    return load_forecasted

## Build NN model

In [ ]:
def make_helper_model(input_shape, lookback, num_services):
    inputs = Input((lookback, input_shape[1], input_shape[2],
                    num_services))
    model = layers.Conv3D(32, (3, 3, 3), activation='relu',
                          padding='same')(inputs)
    model = layers.Conv3D(32, (6, 6, 6), activation='relu',
                          padding='same')(model)
    model = layers.Dropout(0.5)(model, training=True)
    model = layers.Conv3D(16, (6, 6, 6), activation='relu',
                          padding='same')(model)
    model = layers.Dropout(0.5)(model, training=True)
    model = layers.Flatten()(model)
    model = layers.Dense(64, activation='relu')(model)
    model = layers.Dropout(0.3)(model, training=True)
    model = layers.Dense(32, activation='relu')(model)
    output = layers.Dense(num_services, activation='relu')(model)
    model = Model(inputs, output)
    model.compile(optimizer=Adam(0.0005), loss='MAE')
    return model

## Create, training and forecast Block (Helper)

In [ ]:
N = 6
batch_size = 128
num_iters # number of time you repeat the same prediction to measure NN uncertainty
load_input # numpy matrix of shape(len(dataset), nº row, nº col, num_services). 
load_output # numpy matrix of shape (len(dataset), 1, num_services).
model_helper = make_helper_model(load_input.shape, N, num_services)
history = model_helper.fit_generator(train_gen, steps_per_epoch=150, epochs=100, validation_data=val_gen, validation_steps=40, verbose=0)
load_predicted = forecast_batch(model_helper, load_input, min_index=start_index_test_dataset, max_index=end_index_test_dataset, lookback=N, num_services=num_services, num_iters=num_iters)

# (III) Short-term shared capacity allocation

In [ ]:
def return_cplus_fix_ps(p_vector, total_shared, p_s):
    """ Return c_plus given p and max amount of shared for
    that k_o,k_s and time. """
    cplus = np.zeros(p_vector.shape)
    products = np.zeros(p_vector.shape[0])
    for i in range(len(cplus)):
        products[i] = np.true_divide(np.prod(p_vector[i:]),
                                     np.prod((1-p_vector)[i:]))
    cplus_s = np.true_divide(total_shared * p_s,
                            np.sum(products) + 1)
    for i in range(len(cplus)):
        cplus[i] = products[i] * cplus_s
    return cplus, cplus_s

In [ ]:
def cost_func_evaluation_p_fix_ps(p, lambda_i, dedicated, total_shared, p_s, k_o, k_s):
    ''' Evaluate the cost of the allocation given cplus selected.'''
    total_cost = 0
    cplus, cplus_s = return_cplus_fix_ps(p, total_shared, ps)
    for i in range(lambda_i.shape[0]):  # shape[0] = num services
        total_cost += cplus[i] * k_o
        ecdf = ECDF(lambda_i[i])
        total_cost += (1-ecdf(dedicated[i] + cplus[i])) * k_s
    return total_cost

In [ ]:
def return_cplus(p_vector, total_shared):
    """ Return c_plus given p and max amount of shared for
    that phi,alpha and time. """
    cplus = np.zeros(p_vector.shape)
    products = np.zeros(p_vector.shape[0]-1)
    for i in range(len(cplus)-1):
        products[i] = np.true_divide(np.prod(p_vector[i:-1]),
                                     np.prod((1-p_vector)[i:-1]))
    cplus[-1] = np.true_divide(total_shared * p_vector[-1],
                               np.sum(products) + 1)
    for i in range(len(cplus)-1):
        cplus[i] = products[i] * cplus[-1]
    return cplus

In [ ]:
def cost_func_evaluation_p(p, lambda_i, dedicated, total_shared, k_o, k_s):
    ''' Evaluate the cost of the allocation given cplus selected.'''
    total_cost = 0
    cplus = return_cplus(p, total_shared)
    for i in range(lambda_i.shape[0]):  # shape[0] = num services
        total_cost += k_o * cplus[i]
        ecdf = ECDF(forecasting[i])
        total_cost += (1-ecdf(static[i] + cplus[i])) * k_s
    return total_cost

In [ ]:
def fun(p_s, lambda_i, dedicated, total_shared, k_o, k_s, num_services, lower_bound, upper_bound):
    """ Function with only one variable to be minimized through
        bounded golden search
    """
    p_0 = np.ones(num_services) * 0.5
    opt = pybobyqa.solve(cost_func_evaluation_p_fix_ps, p_0,
                         bounds=(lower_bound[:num_services], upper_bound[:num_services]),
                         args=(lambda_i, dedicated, total_shared, p_s, k_o, k_s))
    return opt.f

## Main

In [ ]:
lower_bound = np.ones(num_services) * 1e-10
upper_bound = np.ones(num_services) * 0.99999
# Repeat for each time slot T_s
p0_s = scipy.optimize.minimize_scalar(fun, bounds(1e-10, 0.99999), method='bounded', args=(lambda_i, x_d, x_s, k_o, k_s, num_services, lower_bound, upper_bound))
p0_s = p0_s.x
p0 = np.ones(num_services-1) * 0.5
test = pybobyqa.solve(cost_func_evaluation_p_fix_ps, p0, bounds=(lower_bound[:num_services-1], upper_bound[:num_services-1]),
                      args=(lambda_i, x_d, x_s, p0_s, k_o, k_s))
p0 = np.zeros(num_services)
p0[:num_services-1] = test.x
p0[-1] = p0_s
p = pybobyqa.solve(cost_func_evaluation_p, p0, bounds=(lower_bound[:num_services], upper_bound[:num_services]),
                   args=(lambda_i, x_d, x_s, k_o, k_s))
cplus = return_cplus(p.x, x_s)
p = p.x